<a href="https://colab.research.google.com/github/ELVAN13091999/my-first-binder/blob/main/Data_Science_In_Health_Care_Using_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install